In [94]:
import os
import re
import time
import pickle
from typing import Tuple, List

from smart_open import smart_open
from gensim.utils import simple_preprocess
import spacy
import multiprocessing
#from gensim import 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse
import gensim
from gensim.corpora import Dictionary, MmCorpus
from collections import deque
from collections import namedtuple
import random
from gensim.models.coherencemodel import CoherenceModel
# explore the topics discussed in the reviews and compare how they have evolved over time
# We will define a series of functions to achieve that objective
from gensim.models import phrases
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)

In [2]:
# read the reviews
pitchfork = pd.read_csv('../data/pitchfork/pitchfork.csv', low_memory=False, 
                       encoding = 'utf-8')
pitchfork.head()

,artist,album,genre,score,date,author,role,review,bnm,link,label,release_year
0,David Byrne,“…The Best Live Show of All Time” — NME EP,Rock,5.5,January 11 2019,Andy Beta,Contributor,"Viva Brother, Terris, Mansun, the Twang, Joe L...",0,https://pitchfork.com/reviews/albums/david-byr...,Nonesuch,2018.0
1,DJ Healer,Lost Lovesongs / Lostsongs Vol. 2,Electronic,6.2,January 11 2019,Chal Ravens,Contributor,"The Prince of Denmark—that is, the proper prin...",0,https://pitchfork.com/reviews/albums/dj-healer...,Planet Uterus,2019.0
2,Jorge Velez,Roman Birds,Electronic,7.9,January 10 2019,Philip Sherburne,Contributing Editor,"Jorge Velez has long been prolific, but that’s...",0,https://pitchfork.com/reviews/albums/jorge-vel...,Self-released,2019.0
3,Chandra,Transportation EPs,Rock,7.8,January 10 2019,Andy Beta,Contributor,When the Avalanches returned in 2016 after an ...,0,https://pitchfork.com/reviews/albums/chandra-t...,Telephone Explosion,2018.0
4,The Chainsmokers,Sick Boy,Electronic,3.1,January 9 2019,Larry Fitzmaurice,Contributor,We’re going to be stuck with the Chainsmokers ...,0,https://pitchfork.com/reviews/albums/the-chain...,"Disruptor,Columbia",2018.0


In [3]:
# read the stopwords
with open('../data/pitchfork/stop.txt', 'r') as f:
    stop_words = f.read().split('\n')
stop_words = stop_words+['songs', 'albums','record','records', 'album', 'sound', 'music',
                        'band', 'song', 'time', 'years','fuck', '\\xa0', 'kind','single','shit',
                         'good', '\n', 'great','\xa0', 'sing','rock','bad','guy','lyric',
                        'lot', 'sing','band','rock','man','girl','listen','day','bands','record',
                         'records', 'guitars','thing','pretty','artist','things',
                         'people','stuff','guitar']
#stop_words = set(stop_words)

In [4]:
print(len(pitchfork))
pitchfork['review'] = pitchfork['review'].astype(str)
pitchfork = pitchfork[pitchfork['review'].apply(lambda x: len(x)>200)]
pitchfork.reset_index(inplace=True)
print(len(pitchfork))

20873
20869


In [5]:
#pitchfork['review'] = pitchfork['review'].values.astype('str')
documents = pitchfork['review'].tolist()
doc_ids = pitchfork['link'].tolist()
print(len(documents),len(doc_ids))

20869 20869


In [6]:
documents = [str(doc) for doc in documents if doc]

In [7]:
def tokenize(documents: List[str], stop_words: List[str] = None, use_gpu = True, 
             model = 'en_core_web_lg') -> List[List[str]]:
    if use_gpu:
        spacy.prefer_gpu()
        print(spacy.prefer_gpu())
    # load the model
    nlp = spacy.load(model, disable=['ner', 'parser'])
    # Mark them as stop words
    for word in stop_words:
        nlp.Defaults.stop_words.add(word)
    print(nlp.Defaults.stop_words)
    docs = nlp.pipe(documents, batch_size=256,n_process=multiprocessing.cpu_count()-6)
    docs = [[token.lemma_.lower() for token in doc if not (token.is_stop or token.is_punct or token.is_digit)] for doc in docs]
    #docs = [[token.lemma_ for token in doc] for doc in docs]
    return docs

In [8]:
# create bigrams and create corpus and dictionary to feed into model
def make_bigrams(texts):
    phrases2 = phrases.Phrases(texts, min_count = 10)
    bigram = phrases.Phraser(phrases2)
    return [bigram[doc] for doc in texts]
def show_dfs_topk(docs, topk=20, dictionary = None):
        ndoc = len(docs)
        dfs_topk = sorted([(dictionary.id2token[k],fq) for k,fq in dictionary.dfs.items()], 
                          key=lambda x: x[1],reverse=True)[:topk]
        for i,(word,freq) in enumerate(dfs_topk):
            print(f'{i+1}:{word} --> {freq}/{ndoc} = {(1.0*freq/ndoc):>.13f}')
        return dfs_topk

def topk_dfs(docs, topk=20, dictionary = None):
    ndoc = len(docs)
    dfs_topk = show_dfs_topk(docs, topk=topk, dictionary = dictionary)
    return 1.0*dfs_topk[-1][-1]/ndoc

def show_dfs_topk(docs,topk=20, dictionary = None):
    ndoc = len(docs)
    dfs_topk = sorted([(dictionary.id2token[k],fq) for k,fq in dictionary.dfs.items()],
                      key=lambda x: x[1],reverse=True)[:topk]
    for i,(word,freq) in enumerate(dfs_topk):
        print(f'{i+1}:{word} --> {freq}/{ndoc} = {(1.0*freq/ndoc):>.13f}')
    return dfs_topk

In [9]:
%%time
retokenize = False
if os.path.exists('../data/pitchfork/corpus.mm') and retokenize==False:
    print('tokenization already conducted')
else:
    print('tokenization underway')
    documents_tokenized = tokenize(documents, stop_words = stop_words)
    documents_tokenized = make_bigrams(documents_tokenized)
    #documents_tokenized = [i for i in documents_tokenized if not any(b in stop_words for b in i)]
    documents_tokenized = [[word for word in doc if word not in stop_words] for doc in documents_tokenized]
    dictionary = Dictionary(documents_tokenized)
    print(f'dictionary size is {len(dictionary)}')
    dictionary.id2token = {v:k for k,v in dictionary.token2id.items()} 
    show_dfs_topk(documents_tokenized, topk = 20, dictionary = dictionary)
    ratio = topk_dfs(documents_tokenized, topk=20, dictionary=dictionary)
    print(ratio)
    print(f'before compacting dict is {len(dictionary)} words')
    dictionary.filter_extremes(no_below = 40, no_above = ratio)
    dictionary.compactify()
    print(f'after compacting dict is {len(dictionary)} words')
    # create bows
    bows, docs = [],[]
    for doc in documents_tokenized:
        _bow = dictionary.doc2bow(doc)
        bows.append(_bow)
        docs.append(doc)
    # save the corpuss, dictionary and text
    gensim.corpora.MmCorpus.serialize('../data/pitchfork/corpus.mm', bows)
    dictionary.save_as_text('../data/pitchfork/dict.txt')
    with open('../data/pitchfork/dict.pkl','wb') as f:
        pickle.dump(dictionary,f)
    with open('../data/pitchfork/docs.pkl','wb') as f:
        pickle.dump(docs,f)
    with open('../data/pitchfork/doc_ids.pkl','wb') as f:
        pickle.dump(doc_ids,f)
    vocab_size = len(dictionary)
    num_docs = len(bows)
    print(f'Processed {len(bows)} documents.')
    # we will now create the bow representation

tokenization already conducted
CPU times: user 156 µs, sys: 51 µs, total: 207 µs
Wall time: 149 µs


In [10]:
# load dictionary
dictionary = Dictionary.load('../data/pitchfork/dict.pkl')
bows = gensim.corpora.MmCorpus('../data/pitchfork/corpus.mm')
docs = pickle.load(open('../data/pitchfork/docs.pkl','rb'))
docs_ids = pickle.load(open('../data/pitchfork/doc_ids.pkl','rb'))
print('len of vocabulary is ',len(dictionary))

len of vocabulary is  15048


In [11]:
# print
print(docs[9984], docs_ids[9984])

['closely', 'tie', 'early', 'strange', 'indie', 'hardcore_punk', 'grow', 'early', 'indie', 'art_rocker', 'plenty', 'hardcore', 'late_1980', 'early_90', 'label', 'touch', 'dischord', 'coliseum', 'house', 'curse', 'nod', 'distant_past', 'lp', 'salvation', 'recruit', 'drummer', 'find', 'label', 'move', 'metal', 'monger', 'relapse', 'post', 'rocking', 'temporary_residence', 'opportunity', 'house', 'curse', 'tribute', 'turn', 'the-90s', 'help_shape', 'invite', 'member', 'perform', 'j._robbins', 'jawbox', 'burning_airlines', 'mix', 'add', 'backing_vocal', 'blind', 'eye"--', 'hell', 'track', 'surge', 'rarely', 'hear', 'indie', 'toy', 'quiet_loud', 'dynamic', 'unpredictable', 'rhythm', 'crackle', 'distortion', 'melody', 'straightforward', 'clear', 'patterson', 'discernible', 'matter', 'sputter', 'yowl', 'nuance', 'singing', 'cloak', 'red', 'perimeter', 'subtlety', 'remarkable', 'voice', 'guttural', 'gravel', 'gargle', 'roar', 'john', 'brannon', 'negative_approach', 'laugh', 'hyenas', 'proud', 

In [12]:
# split into training and test
from sklearn.model_selection import train_test_split
x_bows_train, x_bows_test = train_test_split(bows, test_size=0.15, random_state=192)
x_tokens_train, x_tokens_test = train_test_split(docs, test_size=0.15, random_state=192)
x_ids_train, x_ids_test = train_test_split(docs_ids, test_size=0.15, random_state=192)

In [13]:
#item = list(zip(*bows[0]))
#torch.tensor(list(item[1])).float()
print(f'len of training size is {len(x_bows_train)} and tokes are {len(x_tokens_train)} and ids are {len(x_ids_train)}'
      f' len of test size is {len(x_bows_test)} and tokens are {len(x_tokens_test)} and ids are {len(x_ids_test)}')

len of training size is 17738 and tokes are 17738 and ids are 17738 len of test size is 3131 and tokens are 3131 and ids are 3131


In [14]:
from torch.utils.data import Dataset,DataLoader
import torch

class Data_Processing(object):
    def __init__(self, docs, bows, vocab, ids):
        self.docs = docs
        self.bows = bows
        self.vocab = vocab
        self.ids = ids
        
    def __getitem__(self,idx):
        bow = torch.zeros(len(self.vocab))
        # create token and frequency
        item = list(zip(*self.bows[idx])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
        # create
        bow[list(item[0])] = torch.tensor(list(item[1])).float()
        txt = self.docs[idx]
        id_ = self.ids[idx]
        #print(f'shape of bow before being put together in data loader {bow.shape} {type(bow)}')
        return txt, bow, id_
    
    def __len__(self):
        return len(self.docs)
    
    def collate_fn1(self,batch_data):
        texts,bows,id_ = list(zip(*batch_data))
        #print(f'what happens with collate function {torch.stack(bows,dim=0)}, {torch.stack(bows,dim=0).shape}')
        return texts,torch.stack(bows,dim=0),id_

    def __iter__(self):
        for doc in self.docs:
            yield doc

batch_size = 512

# create a class to process the traininga and test data
training_data = Data_Processing(x_tokens_train, x_bows_train, dictionary, x_ids_train)
test_data = Data_Processing(x_tokens_test, x_bows_test, dictionary, x_ids_test)

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1),
                    'test': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=test_data.collate_fn1)}
dataset_sizes = {'train':len(training_data)}
example = next(iter(dataloaders_dict.get('train')))

In [15]:
# check the dataloader works
print(example[2][0],example[0][0])

https://pitchfork.com/reviews/albums/4362-the-life-aquatic-studio-sessions-featuring-seu-jorge/ ['crawl', 'subway_car', 'home', 'work', 'loud', 'scraggly', 'blare', 'lady_gentleman', 'minute', 'absolutely', 'question', 'polite', 'person', 'leg', 'wheelchair', 'acoustic', 'strapped', 'insist', 'perform', 'keenly', 'request', 'wink', 'wink', 'donation', 'tune', 'world', 'studebaker', 'sake', 'identification', 'popular', 'baby', 'feeling', 'bootylicious', 'replace', 'stoopid', 'clever', 'highlight', 'plight', 'subway_car', 'handle', 'line', 'handle', 'america', 'handle', 'handle', 'buck', 'seu_jorge', 'entrance', 'hipster', 'consciousness', 'position', 'studebaker', 'tale', 'jorge', 'fashion', 'david_bowie', 'performance', 'wes_anderson', 'life', 'aquatic', 'hat--', 'cute', 'dvd', 'commentary', 'story', 'bar', 'stool', 'chat', 'point', 'fat', 'baby', 'lyrical', 'rejigger', 'couple', 'fact', 'strip', 'hunky', 'stardust', 'era', 'glamthem', 'smoky', 'bossa_nova', 'make', 'regular', 'subway'

In [16]:
# define the architechture
import torch
import torch.nn.functional as F
from torch import nn


class ETM(nn.Module):
    # instantiate the model
    def __init__(self, device = None, num_topics = 20, vocab_size = None, t_hidden_size = 1024, 
                 rho_size = 300, emb_size=None, theta_act='relu',
                 embeddings=None, train_embeddings=True, enc_drop=0.5):
        '''
        Creates an embedded topic model instance. The model hyperparameters are:
        device: if using cuda or cpu
        num_topics: number of topics to be used
        vocab_size: size of the vocabulary
        t_hidden_size: size of the hidden embedding   
        embeddings (str or KeyedVectors): KeyedVectors instance containing word-vector mapping for embeddings, 
                                          or its path. None by default
        rho_size (int): dimension of rho (embeddings vector)
        emb_size (int): dimens-ion of embeddings if not trained jointly, default to None
        theta_act (str): activation function can be tanh, softplus, relu, rrelu, leakyrelu, elu, selu, glu, etc.
        train_embeddings (bool): whether to fix rho or train it, defaults to True
        enc_drop: dropout value for activation function, defaults to 0.5
        '''
        super(ETM, self).__init__()
        # define hyperparameters
        self.num_topics = num_topics
        self.vocab_size = vocab_size
        # in case we want to train the embeddings this parameter controls the hidden size of the embeddings
        self.rho_size = rho_size
        # dimension of the matrix for the topic mixture (topics hidden size)
        self.t_hidden_size = t_hidden_size
        # dropout to be applied to the encoder
        self.enc_drop = enc_drop
        # size of the embeddings in case a pretrained embedding is provided
        self.emb_size = emb_size
        # dropout rate
        self.t_drop = nn.Dropout(enc_drop)
        #
        self.train_embeddings = train_embeddings
        
        # activation for theta
        self.theta_act = self.get_activation(theta_act)
        # device to use for training
        self.device = device

        # define the word embedding matrix \rho
        if self.train_embeddings:
            self.rho = nn.Linear(rho_size, vocab_size, bias=True)
        else:
            # get the dimension of the embeddings
            print(embeddings.size())
            num_embeddings, emb_size = embeddings.size()
            # instantiate rho embeddings
            self.rho = embeddings.clone().float().to(self.device)

        # define the matrix containing the topic embeddings
        self.alphas = nn.Linear(rho_size, num_topics, bias=False)

        # define variational distribution for \theta_{1:D} via amortization
        # this takes the vocabulary size, creates the hidden size for the
        # topics matrix, applies activation function, applies another linear
        # layer followed by additional activation function
        self.q_theta = nn.Sequential(nn.Linear(vocab_size, t_hidden_size),
            self.theta_act,
            nn.Linear(t_hidden_size, t_hidden_size),
            self.theta_act,
        )
        
        # linear transformation of the topics hidden size to the number of topics
        self.mu_q_theta = nn.Linear(t_hidden_size, num_topics, bias=True)
        
        # another linear transformation from hidden size to number of topics
        self.logsigma_q_theta = nn.Linear(t_hidden_size, num_topics, bias=True)

    def get_activation(self, act = 'relu'):
        '''
        Define the activation function options available are tahn, relu, softplus, rrelu, 
        leakyrelu, elu, selu and glu.
        '''
        if act == 'tanh':
            act = nn.Tanh()
        elif act == 'relu':
            act = nn.ReLU()
        elif act == 'softplus':
            act = nn.Softplus()
        elif act == 'rrelu':
            act = nn.RReLU()
        elif act == 'leakyrelu':
            act = nn.LeakyReLU()
        elif act == 'elu':
            act = nn.ELU()
        elif act == 'selu':
            act = nn.SELU()
        elif act == 'glu':
            act = nn.GLU()
        else:
            act = nn.Tanh()
        return act

    def reparameterize(self, mu, logvar):
        """
        Returns a sample from a Gaussian distribution via reparameterization.
        mu stands for
        logvar stands for 
        reference for reparametrizetion trick. 
        https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important
        """
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul_(std).add_(mu)
        else:
            return mu

    def encode(self, bows):
        """Returns paramters of the variational distribution for \theta.
        input: bows batch of bag-of-words...tensor of shape bsz x V (batch size* vocabulary)
        output: mu_theta, log_sigma_theta
        """
        
        q_theta = self.q_theta(bows)
        if self.enc_drop > 0:
            q_theta = self.t_drop(q_theta)
        # mu_q_theta takes the topic hidden size and outputs a vector with the dimension of the topics
        mu_theta = self.mu_q_theta(q_theta)
        # takes the topic hidden size and outpus another vector equal to the size of the vectors
        logsigma_theta = self.logsigma_q_theta(q_theta)
        # returns kl theta which is a normalized number
        kl_theta = -0.5 * torch.sum(1 + logsigma_theta - mu_theta.pow(2) - logsigma_theta.exp(), dim=-1).mean()
        #print(f'this is kl_theta {kl_theta}')
        return mu_theta, logsigma_theta, kl_theta
       
    def get_beta(self):
        """
        This generate the description as a defintion over words
        Returns:
            [type]: [description]
        """
        try:
            logit = self.alphas(self.rho.weight) # torch.mm(self.rho, self.alphas)
        except:
            logit = self.alphas(self.rho)
        # gets logits over vocabulary dimension when fitting to the topic
        beta = F.softmax(logit, dim=0).transpose(1, 0) ## softmax over vocab dimension
        #print(f'this is beta {beta}')
        return beta

    def get_theta(self, normalized_bows):
        """
        getting the topic poportion for the document passed in the normalixe bow or tf-idf
        """
        # gets mu_theta (linear layer for topic dimension), logsima_theta another liner layer output
        # and the kld_tjet
        mu_theta, logsigma_theta, kld_theta = self.encode(normalized_bows)
        # gets variable to backpropagate, returns  reparametized mu theta
        z = self.reparameterize(mu_theta, logsigma_theta)
        # applied softmax and returns squeezed logits
        theta = F.softmax(z, dim=-1) 
        #print(f'this is theta {theta}')
        return theta, kld_theta

    def decode(self, theta, beta):
        """compute the probability of topic given the document which is equal to theta^T ** B
        Args:
            theta ([type]): vector of proabilities of belonging to a given topic
            beta ([type]): probability of belonging to a given topic
        Returns:
            [type]: [description]
        """
        # multiplication of softmax with beta (likelihood of belonging to a topic given likelihood a word
        # belongs to another topic)
        res = torch.mm(theta, beta)
        # normalizes predictions
        almost_zeros = torch.full_like(res, 1e-6)
        results_without_zeros = res.add(almost_zeros)
        predictions = torch.log(results_without_zeros)
        return predictions

    def forward(self, bows, normalized_bows, theta=None, aggregate=True):
        # get \theta
        if theta is None:
            theta, kld_theta = self.get_theta(normalized_bows)
        else:
            kld_theta = None

        # get \beta
        beta = self.get_beta()

        # get prediction loss
        preds = self.decode(theta, beta)
        recon_loss = -(preds * bows).sum(1)
        if aggregate:
            recon_loss = recon_loss.mean()
        return recon_loss, kld_theta

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [18]:
# instantiate the model
model = ETM(device = device, num_topics = 20, vocab_size =len(dictionary), 
            t_hidden_size = 1024, rho_size = 400, 
            emb_size=None, theta_act='tahn',
            embeddings=None, train_embeddings=True, enc_drop=0.5)

In [19]:
model

ETM(
  (t_drop): Dropout(p=0.5, inplace=False)
  (theta_act): Tanh()
  (rho): Linear(in_features=400, out_features=15048, bias=True)
  (alphas): Linear(in_features=400, out_features=20, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=15048, out_features=1024, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Tanh()
  )
  (mu_q_theta): Linear(in_features=1024, out_features=20, bias=True)
  (logsigma_q_theta): Linear(in_features=1024, out_features=20, bias=True)
)

In [20]:
# define the optimizer
from torch.optim import lr_scheduler, AdamW, SGD,Adam
lr = 0.003
optimizer = Adam(model.parameters(), lr=lr, weight_decay = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.003
    weight_decay: 0.001
)

In [21]:
#model.forward()
def get_topics(model = None, num_topics = None, top_n_words=10, vocabulary = None) -> List[str]:
        """
        Returns the topics of the model with the specified number of words
        mode: the trained ETM model
        num_topics: The number of topics
        top_n_words: the number of words to return per topic
        vocabulary: dictionary with the vocabularty used to train the model
        """
        # set the model to eval
        with torch.no_grad():
            # instantiate the list to save the word list of the models
            topics = []
            # get the topic word distribution
            gammas = model.get_beta()
            # iterate over the topic distributions to get per topic word distribution
            for k in range(model.num_topics):
                gamma = gammas[k]
                top_words = list(gamma.cpu().numpy().argsort()
                                 [-top_n_words:][::-1])
                topic_words = [vocabulary[a] for a in top_words]
                topics.append(topic_words)

            return topics

def get_topic_words(model,dictionary):
    """
    Returns the topics of the model with all the words of the topic
    model: the trained ETM model
    dictionary: dictionary with the vocabulary used to train the model
        """
    with torch.no_grad():
        topic_words = []
        gammas = model.get_beta()
        
        for k in range(model.num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[::-1])
            tp = [dictionary[a] for a in top_words]
            topic_words.append(tp)
    
    return topic_words
        
def calc_topic_diversity(topic_words):
    """
    Function to calculate topic diversity
    original function from: https://github.com/zll17/Neural_Topic_Models/blob/master/utils.py
    """
    vocab = set(sum(topic_words,[]))
    n_total = len(topic_words) * len(topic_words[0])
    print(len(topic_words))
    topic_div = len(vocab) / n_total
    return topic_div


def get_topic_diversity(model, topk=50):
    """
    
    original function from https://github.com/adjidieng/ETM/blob/master/utils.py
    """
    with torch.no_grad():
        beta = model.get_beta()
        num_topics = model.num_topics
        print(beta.shape, num_topics)
        list_w = np.zeros((num_topics, topk))
        print(list_w.shape)
        for k in range(num_topics):
            idx = beta[k,:].cpu().numpy().argsort()[-topk:][::-1]
            list_w[k,:] = idx
        n_unique = len(np.unique(list_w))
        TD = n_unique / (topk * num_topics)
        #print('Topic diveristy is: {}'.format(TD))
        return TD
    
def coherence_data(topics = None, texts = None, dictionary = None):
    c_uci_coherence_model = CoherenceModel(topics=topics,texts=docs,dictionary=dictionary,coherence='c_uci')
    c_uci_score = c_uci_coherence_model.get_coherence()
    c_npmi_coherence_model = CoherenceModel(topics=topics,texts=docs,dictionary=dictionary,coherence='c_npmi')
    c_npmi_score = c_npmi_coherence_model.get_coherence()
    return c_uci_score,c_npmi_score


In [84]:
from pprint import pprint
def nearest_neighbors(word, embeddings, vocab, n_most_similar=5):
    """
    original function from https://github.com/lffloyd/embedded-topic-model/blob/main/embedded_topic_model/utils/metrics.py
    """
    
    vectors = embeddings.data.cpu().numpy()
    index = dictionary.token2id[word]
    query = vectors[index]
    ranks = vectors.dot(query).squeeze()
    denom = query.T.dot(query).squeeze()
    denom = denom * np.sum(vectors**2, 1)
    denom = np.sqrt(denom)
    ranks = ranks / denom
    mostSimilar = []
    [mostSimilar.append(idx) for idx in ranks.argsort()[::-1]]
    nearest_neighbors = mostSimilar[1:n_most_similar]
    nearest_neighbors = [vocab[comp] for comp in nearest_neighbors]
    return nearest_neighbors

def get_most_similar_words(model = None, queries=[], vocabulary = None, n_most_similar=5) -> dict:
        """
        Gets the nearest neighborhoring words for a list of tokens. By default, returns the 20 most similar words for each token in 'queries' array.
        Parameters:
        ===
            queries (list of str): words to find similar ones
            n_most_similar (int): number of most similar words to get for each word given in the input. By default is 20
        Returns:
        ===
            dict of (str, list of str): dictionary containing the mapping between query words given and their respective similar words
        """

        model.eval()

        # visualize word embeddings by using V to get nearest neighbors
        with torch.no_grad():
            embeddings = model.rho.weight  # Vocab_size x E
            

            neighbors = {}
            for word in queries:
                neighbors[word] = nearest_neighbors(
                    word,embeddings, vocabulary, n_most_similar)

            return neighbors
        
def document_inference(model = None, dataloader = None, dictionary = None):
    topic_list = []
    document_id = []
    with torch.no_grad():
        for iter_,data in enumerate(dataloader):
            txts,bows,ids = data
            # send bows to device
            bows = bows.to(device)
            # get topics
            topics,_ = model.get_theta(bows)
            # send to cpu
            t = topics.clone().detach().cpu().numpy()
            # append list to documents
            topic_list.append(t.tolist())
            # get ids
            document_id.append(ids)
    # create data frame
    columns_topics = []
    for i in range(model.num_topics):
        column = 'topic '+str(i)
        columns_topics.append(column)
    topic_list = [item for sublist in topic_list for item in sublist]
    document_id = [item for sublist in document_id for item in sublist]
    topic_dataframe = pd.DataFrame(topic_list,
                                   columns = columns_topics)
    id_data = pd.DataFrame(document_id, columns=['document_id'])
    dataframe = pd.concat([id_data,topic_dataframe], axis = 1)
    return dataframe

In [23]:
import wandb
wandb.init(entity="jlealtru", project="ETM_runs_p")
config = wandb.config          # Initialize config

def train(model, batch_size=256,dictionary = None,
          learning_rate=2e-3,test_data=None,
          num_epochs=600,is_evaluate=False,log_every=40,ckpt=None):
    model.to(device)
    model.train()
    
    data_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    
    
    if ckpt:
        self.load_model(ckpt["net"])
        optimizer.load_state_dict(ckpt["optimizer"])
        start_epoch = ckpt["epoch"] + 1
    else:
        start_epoch = 0
    
    acc_loss = 0
    acc_kl_theta_loss = 0
    cnt = 0
    
    trainloss_lst, valloss_lst = [], []
    recloss_lst, klloss_lst = [],[]
    c_v_lst, c_w2v_lst, c_uci_lst, c_npmi_lst, mimno_tc_lst, td_lst = [], [], [], [], [], []
    for epoch in range(start_epoch, num_epochs):
        epochloss_lst = []
        model.train()
        for iter_,data in enumerate(data_loader):
            #optimizer.zero_grad()
            model.zero_grad(set_to_none=True)
            
            txts,bows,ids = data
            bows = bows.to(device)
            normalized_bows = bows
            normalized_bows.to(device)
            
            recon_loss, kld_theta = model.forward(bows, normalized_bows)
            total_loss = recon_loss + kld_theta
            total_loss.backward()
            optimizer.step()

            acc_loss += torch.sum(recon_loss).item()
            acc_kl_theta_loss += torch.sum(kld_theta).item()
            cnt += 1
            if iter_ % 4 == 0:
                cur_loss = round(acc_loss / cnt, 2) 
                cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
                cur_real_loss = round(cur_loss + cur_kl_theta, 2)

                print('Epoch: {} KL_theta: is {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, cur_kl_theta, cur_loss, cur_real_loss))
                
                # add wandb
                wandb.log({"Epoch": epoch,
                           "Train Loss": cur_kl_theta,"rec_loss": cur_loss,'NELBO':cur_real_loss,
                          "lr": learning_rate,
                          "optimizer": 'Adam'})

        cur_loss = round(acc_loss / cnt, 2) 
        cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
        cur_real_loss = round(cur_loss + cur_kl_theta, 2)
        print('*'*100)
        print('Epoch: {} KL_theta: is {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, cur_kl_theta, cur_loss, cur_real_loss))
        if (epoch+1)%log_every==0:
            topic_words = get_topic_words(model, dictionary)
            topic_diversity = get_topic_diversity(model,topk=200)
            coh_scores = coherence_data(topics = topic_words, texts = txts, dictionary = dictionary)
            #calc_topic_diversity(topic_words)
            print(f'topic diversity is {topic_diversity}')
            pprint(get_topics(model = model, num_topics = model.num_topics, top_n_words= 10, vocabulary = dictionary))
            wandb.log({"Epoch": epoch,
                       "Train Loss": cur_kl_theta,"rec_loss": cur_loss,'NELBO':cur_real_loss,
                       "lr": learning_rate,
                       "optimizer": 'Adam',
                      'topic_diversity': topic_diversity,
                       'uci': coh_scores[0],
                       'npmi':coh_scores[1]
                      })

wandb: Currently logged in as: jlealtru (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [24]:
#model.to(device)
wandb.watch(model, log="all")
train(model,batch_size=1024, learning_rate=2e-3,test_data=None,dictionary=dictionary,
      num_epochs=800,is_evaluate=False,log_every=40,
      ckpt=None)

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch: 0 KL_theta: is 0.03 .. Rec_loss: 2155.05 .. NELBO: 2155.08
Epoch: 0 KL_theta: is 7.19 .. Rec_loss: 2158.19 .. NELBO: 2165.38
Epoch: 0 KL_theta: is 5.12 .. Rec_loss: 2138.74 .. NELBO: 2143.86
Epoch: 0 KL_theta: is 4.24 .. Rec_loss: 2134.33 .. NELBO: 2138.57
Epoch: 0 KL_theta: is 3.56 .. Rec_loss: 2127.1 .. NELBO: 2130.66
****************************************************************************************************
Epoch: 0 KL_theta: is 3.4 .. Rec_loss: 2123.75 .. NELBO: 2127.15
Epoch: 1 KL_theta: is 3.27 .. Rec_loss: 2120.68 .. NELBO: 2123.95
Epoch: 1 KL_theta: is 2.91 .. Rec_loss: 2106.98 .. NELBO: 2109.89
Epoch: 1 KL_theta: is 2.61 .. Rec_loss: 2091.5 .. NELBO: 2094.11
Epoch: 1 KL_theta: is 2.35 .. Rec_loss: 2078.17 .. NELBO: 2080.52
Epoch: 1 KL_theta: is 2.11 .. Rec_loss: 2065.36 .. NELBO: 2067.47
****************************************************************************************************
Epoch: 1 KL_theta: is 2.06 .. Rec_loss: 2063.14 .. NELBO: 2065.2
Epoch: 2 K

****************************************************************************************************
Epoch: 16 KL_theta: is 0.7 .. Rec_loss: 1963.87 .. NELBO: 1964.57
Epoch: 17 KL_theta: is 0.7 .. Rec_loss: 1963.85 .. NELBO: 1964.55
Epoch: 17 KL_theta: is 0.72 .. Rec_loss: 1963.49 .. NELBO: 1964.21
Epoch: 17 KL_theta: is 0.74 .. Rec_loss: 1963.34 .. NELBO: 1964.08
Epoch: 17 KL_theta: is 0.76 .. Rec_loss: 1963.12 .. NELBO: 1963.88
Epoch: 17 KL_theta: is 0.78 .. Rec_loss: 1962.93 .. NELBO: 1963.71
****************************************************************************************************
Epoch: 17 KL_theta: is 0.78 .. Rec_loss: 1962.83 .. NELBO: 1963.61
Epoch: 18 KL_theta: is 0.79 .. Rec_loss: 1962.7 .. NELBO: 1963.49
Epoch: 18 KL_theta: is 0.81 .. Rec_loss: 1962.33 .. NELBO: 1963.14
Epoch: 18 KL_theta: is 0.83 .. Rec_loss: 1962.07 .. NELBO: 1962.9
Epoch: 18 KL_theta: is 0.85 .. Rec_loss: 1961.93 .. NELBO: 1962.78
Epoch: 18 KL_theta: is 0.87 .. Rec_loss: 1961.86 .. NELBO: 1962.7

Epoch: 33 KL_theta: is 2.11 .. Rec_loss: 1950.83 .. NELBO: 1952.94
Epoch: 33 KL_theta: is 2.12 .. Rec_loss: 1950.58 .. NELBO: 1952.7
Epoch: 33 KL_theta: is 2.14 .. Rec_loss: 1950.59 .. NELBO: 1952.73
Epoch: 33 KL_theta: is 2.15 .. Rec_loss: 1950.52 .. NELBO: 1952.67
****************************************************************************************************
Epoch: 33 KL_theta: is 2.16 .. Rec_loss: 1950.44 .. NELBO: 1952.6
Epoch: 34 KL_theta: is 2.16 .. Rec_loss: 1950.43 .. NELBO: 1952.59
Epoch: 34 KL_theta: is 2.18 .. Rec_loss: 1950.36 .. NELBO: 1952.54
Epoch: 34 KL_theta: is 2.2 .. Rec_loss: 1950.2 .. NELBO: 1952.4
Epoch: 34 KL_theta: is 2.21 .. Rec_loss: 1950.08 .. NELBO: 1952.29
Epoch: 34 KL_theta: is 2.23 .. Rec_loss: 1949.91 .. NELBO: 1952.14
****************************************************************************************************
Epoch: 34 KL_theta: is 2.23 .. Rec_loss: 1949.93 .. NELBO: 1952.16
Epoch: 35 KL_theta: is 2.24 .. Rec_loss: 1949.9 .. NELBO: 1952.14


Epoch: 45 KL_theta: is 2.96 .. Rec_loss: 1945.33 .. NELBO: 1948.29
Epoch: 45 KL_theta: is 2.98 .. Rec_loss: 1945.22 .. NELBO: 1948.2
Epoch: 45 KL_theta: is 2.99 .. Rec_loss: 1945.11 .. NELBO: 1948.1
Epoch: 45 KL_theta: is 3.01 .. Rec_loss: 1945.05 .. NELBO: 1948.06
Epoch: 45 KL_theta: is 3.03 .. Rec_loss: 1945.01 .. NELBO: 1948.04
****************************************************************************************************
Epoch: 45 KL_theta: is 3.03 .. Rec_loss: 1944.99 .. NELBO: 1948.02
Epoch: 46 KL_theta: is 3.03 .. Rec_loss: 1944.99 .. NELBO: 1948.02
Epoch: 46 KL_theta: is 3.05 .. Rec_loss: 1944.77 .. NELBO: 1947.82
Epoch: 46 KL_theta: is 3.07 .. Rec_loss: 1944.72 .. NELBO: 1947.79
Epoch: 46 KL_theta: is 3.08 .. Rec_loss: 1944.67 .. NELBO: 1947.75
Epoch: 46 KL_theta: is 3.1 .. Rec_loss: 1944.6 .. NELBO: 1947.7
****************************************************************************************************
Epoch: 46 KL_theta: is 3.1 .. Rec_loss: 1944.59 .. NELBO: 1947.69


...

Epoch: 740 KL_theta: is 11.57 .. Rec_loss: 1904.94 .. NELBO: 1916.51
Epoch: 740 KL_theta: is 11.57 .. Rec_loss: 1904.94 .. NELBO: 1916.51
Epoch: 740 KL_theta: is 11.57 .. Rec_loss: 1904.94 .. NELBO: 1916.51
Epoch: 740 KL_theta: is 11.57 .. Rec_loss: 1904.93 .. NELBO: 1916.5
Epoch: 740 KL_theta: is 11.57 .. Rec_loss: 1904.93 .. NELBO: 1916.5
****************************************************************************************************
Epoch: 740 KL_theta: is 11.57 .. Rec_loss: 1904.93 .. NELBO: 1916.5
Epoch: 741 KL_theta: is 11.57 .. Rec_loss: 1904.92 .. NELBO: 1916.49
Epoch: 741 KL_theta: is 11.57 .. Rec_loss: 1904.92 .. NELBO: 1916.49
Epoch: 741 KL_theta: is 11.57 .. Rec_loss: 1904.92 .. NELBO: 1916.49
Epoch: 741 KL_theta: is 11.57 .. Rec_loss: 1904.92 .. NELBO: 1916.49
Epoch: 741 KL_theta: is 11.57 .. Rec_loss: 1904.92 .. NELBO: 1916.49
****************************************************************************************************
Epoch: 741 KL_theta: is 11.57 .. Rec_loss:

Epoch: 756 KL_theta: is 11.59 .. Rec_loss: 1904.81 .. NELBO: 1916.4
Epoch: 756 KL_theta: is 11.59 .. Rec_loss: 1904.81 .. NELBO: 1916.4
Epoch: 756 KL_theta: is 11.59 .. Rec_loss: 1904.81 .. NELBO: 1916.4
Epoch: 756 KL_theta: is 11.59 .. Rec_loss: 1904.81 .. NELBO: 1916.4
Epoch: 756 KL_theta: is 11.59 .. Rec_loss: 1904.81 .. NELBO: 1916.4
****************************************************************************************************
Epoch: 756 KL_theta: is 11.59 .. Rec_loss: 1904.8 .. NELBO: 1916.39
Epoch: 757 KL_theta: is 11.59 .. Rec_loss: 1904.8 .. NELBO: 1916.39
Epoch: 757 KL_theta: is 11.59 .. Rec_loss: 1904.8 .. NELBO: 1916.39
Epoch: 757 KL_theta: is 11.59 .. Rec_loss: 1904.8 .. NELBO: 1916.39
Epoch: 757 KL_theta: is 11.59 .. Rec_loss: 1904.8 .. NELBO: 1916.39
Epoch: 757 KL_theta: is 11.6 .. Rec_loss: 1904.8 .. NELBO: 1916.4
****************************************************************************************************
Epoch: 757 KL_theta: is 11.6 .. Rec_loss: 1904.79 ..

Epoch: 767 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
****************************************************************************************************
Epoch: 767 KL_theta: is 11.61 .. Rec_loss: 1904.73 .. NELBO: 1916.34
Epoch: 768 KL_theta: is 11.61 .. Rec_loss: 1904.73 .. NELBO: 1916.34
Epoch: 768 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
Epoch: 768 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
Epoch: 768 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
Epoch: 768 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
****************************************************************************************************
Epoch: 768 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
Epoch: 769 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
Epoch: 769 KL_theta: is 11.61 .. Rec_loss: 1904.72 .. NELBO: 1916.33
Epoch: 769 KL_theta: is 11.61 .. Rec_loss: 1904.71 .. NELBO: 1916.32
Epoch: 769 KL_theta: is 11.61 .. Rec_lo

Epoch: 783 KL_theta: is 11.63 .. Rec_loss: 1904.62 .. NELBO: 1916.25
****************************************************************************************************
Epoch: 783 KL_theta: is 11.63 .. Rec_loss: 1904.62 .. NELBO: 1916.25
Epoch: 784 KL_theta: is 11.63 .. Rec_loss: 1904.62 .. NELBO: 1916.25
Epoch: 784 KL_theta: is 11.63 .. Rec_loss: 1904.62 .. NELBO: 1916.25
Epoch: 784 KL_theta: is 11.63 .. Rec_loss: 1904.62 .. NELBO: 1916.25
Epoch: 784 KL_theta: is 11.64 .. Rec_loss: 1904.61 .. NELBO: 1916.25
Epoch: 784 KL_theta: is 11.64 .. Rec_loss: 1904.61 .. NELBO: 1916.25
****************************************************************************************************
Epoch: 784 KL_theta: is 11.64 .. Rec_loss: 1904.62 .. NELBO: 1916.26
Epoch: 785 KL_theta: is 11.64 .. Rec_loss: 1904.61 .. NELBO: 1916.25
Epoch: 785 KL_theta: is 11.64 .. Rec_loss: 1904.61 .. NELBO: 1916.25
Epoch: 785 KL_theta: is 11.64 .. Rec_loss: 1904.61 .. NELBO: 1916.25
Epoch: 785 KL_theta: is 11.64 .. Rec_lo

Epoch: 799 KL_theta: is 11.66 .. Rec_loss: 1904.51 .. NELBO: 1916.17
****************************************************************************************************
Epoch: 799 KL_theta: is 11.66 .. Rec_loss: 1904.51 .. NELBO: 1916.17
torch.Size([20, 15048]) 20
(20, 200)
topic diversity is 0.3565
[['r&b',
  'singer',
  'soul',
  'hit',
  'producer',
  'year',
  'dance',
  'prince',
  'production',
  'debut'],
 ['kid',
  'joke',
  'party',
  'fun',
  'boy',
  'call',
  'funny',
  'fucking',
  'sex',
  'movie'],
 ['metal',
  'riff',
  'black_metal',
  'doom',
  'heavy',
  'death',
  'black',
  'drum',
  'hardcore',
  'noise'],
 ['life',
  'word',
  'line',
  'write',
  'world',
  'feeling',
  'leave',
  'death',
  'light',
  'dream'],
 ['world',
  'life',
  'black',
  'political',
  'woman',
  'write',
  'word',
  'power',
  'war',
  'call'],
 ['country',
  'blue',
  'folk',
  'cover',
  'write',
  'dylan',
  'acoustic',
  'oldham',
  'american',
  'solo'],
 ['anderson',
  'set',
  '

In [25]:
# save model
torch.save(model.state_dict(), '../data/pitchfork/etm_original_architecture_400d.pth')

In [27]:
# reload model and do inference
# load for inference
model = ETM(device = device, num_topics = 20, vocab_size =len(dictionary), 
            t_hidden_size = 1024, rho_size = 400, 
            emb_size=None, theta_act='tahn',
            embeddings=None, train_embeddings=True, enc_drop=0.5)
model.load_state_dict(torch.load('../data/pitchfork/etm_original_architecture_400d.pth'))
model.to(device)

ETM(
  (t_drop): Dropout(p=0.5, inplace=False)
  (theta_act): Tanh()
  (rho): Linear(in_features=400, out_features=15048, bias=True)
  (alphas): Linear(in_features=400, out_features=20, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=15048, out_features=1024, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Tanh()
  )
  (mu_q_theta): Linear(in_features=1024, out_features=20, bias=True)
  (logsigma_q_theta): Linear(in_features=1024, out_features=20, bias=True)
)

In [29]:
# get the most similar word
get_most_similar_words(model = model, queries=['radiohead'], vocabulary = dictionary, n_most_similar=5)

{'radiohead': ['indie_scene', 'tinkering', 'thom_yorke', 'mercury_prize']}

In [106]:
# get topic words
topics = get_topics(model = model, num_topics = model.num_topics, top_n_words= 20, vocabulary = dictionary)
for i in range(len(topics)):
    print(f'topic {i}: {topics[i]}')

topic 0: ['r&b', 'singer', 'soul', 'hit', 'producer', 'year', 'dance', 'prince', 'production', 'debut', 'star', 'synth', 'big', 'produce', 'woman', 'ballad', 'feature', 'funk', 'world', 'video']
topic 1: ['kid', 'joke', 'party', 'fun', 'boy', 'call', 'funny', 'fucking', 'sex', 'movie', 'talk', 'friend', 'wolf', 'white', 'soundtrack', 'cover', 'laugh', 'title', 'ass', 'start']
topic 2: ['metal', 'riff', 'black_metal', 'doom', 'heavy', 'death', 'black', 'drum', 'hardcore', 'noise', 'suggest', 'lead', 'slow', 'dark', 'start', 'scream', 'past', 'year', 'solo', 'offer']
topic 3: ['life', 'word', 'line', 'write', 'world', 'feeling', 'leave', 'death', 'light', 'dream', 'story', 'place', 'heart', 'relationship', 'lose', 'dark', 'sense', 'home', 'emotional', 'emotion']
topic 4: ['world', 'life', 'black', 'political', 'woman', 'write', 'word', 'power', 'war', 'call', 'art', 'live', 'america', 'message', 'year', 'personal', 'american', 'politic', 'speak', 'child']
topic 5: ['country', 'blue', 'fo

In [102]:
documents_with_topics_test = document_inference(model = model, dataloader = dataloaders_dict.get('test'), dictionary=dictionary)

In [103]:
documents_with_topics_test.head(5)

,document_id,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,topic 11,topic 12,topic 13,topic 14,topic 15,topic 16,topic 17,topic 18,topic 19
0,https://pitchfork.com/reviews/albums/3363-skeleton/,0.020002,0.016442,0.021690,0.016183,0.015497,0.022341,0.023103,0.012285,0.020211,0.054780,0.241943,0.017399,0.034768,0.029451,0.380438,0.009850,0.011421,0.018996,0.016946,0.016255
1,https://pitchfork.com/reviews/albums/7050-the-ride/,0.026106,0.041788,0.020311,0.031221,0.016598,0.010298,0.011094,0.021743,0.070640,0.238277,0.012254,0.014024,0.023244,0.303066,0.022850,0.017588,0.067597,0.020817,0.018023,0.012462
2,https://pitchfork.com/reviews/albums/4557-minimum-maximum/,0.011536,0.017108,0.013754,0.014633,0.022131,0.008629,0.014069,0.076128,0.007191,0.023601,0.009261,0.010604,0.021526,0.014054,0.024437,0.143584,0.091663,0.018222,0.023444,0.434423
3,https://pitchfork.com/reviews/albums/11418-going-places-the-august-darnell-years-1976-1983/,0.087805,0.056293,0.029838,0.016234,0.020594,0.023717,0.034106,0.358788,0.028726,0.033581,0.030064,0.018855,0.019831,0.013588,0.021222,0.052132,0.011995,0.013087,0.085314,0.044230
4,https://pitchfork.com/reviews/albums/21145-meow-the-jewels/,0.019409,0.087558,0.015504,0.024806,0.055348,0.012996,0.014934,0.050035,0.031041,0.007007,0.008545,0.459265,0.021553,0.022175,0.020627,0.060883,0.028470,0.016778,0.015102,0.027964


In [104]:
# merge with original data
documents_with_topics_test = pd.merge(documents_with_topics_test,pitchfork[['artist','album','score','genre','link']],how='left', right_on = 'link',
                                      left_on = 'document_id')
documents_with_topics_test.head()

,document_id,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,topic 11,topic 12,topic 13,topic 14,topic 15,topic 16,topic 17,topic 18,topic 19,artist,album,score,genre,link
0,https://pitchfork.com/reviews/albums/3363-skeleton/,0.020002,0.016442,0.021690,0.016183,0.015497,0.022341,0.023103,0.012285,0.020211,0.054780,0.241943,0.017399,0.034768,0.029451,0.380438,0.009850,0.011421,0.018996,0.016946,0.016255,Figurines,Skeleton,8.3,Rock,https://pitchfork.com/reviews/albums/3363-skeleton/
1,https://pitchfork.com/reviews/albums/7050-the-ride/,0.026106,0.041788,0.020311,0.031221,0.016598,0.010298,0.011094,0.021743,0.070640,0.238277,0.012254,0.014024,0.023244,0.303066,0.022850,0.017588,0.067597,0.020817,0.018023,0.012462,Seaworthy,The Ride,8.1,"Electronic,Rock",https://pitchfork.com/reviews/albums/7050-the-ride/
2,https://pitchfork.com/reviews/albums/4557-minimum-maximum/,0.011536,0.017108,0.013754,0.014633,0.022131,0.008629,0.014069,0.076128,0.007191,0.023601,0.009261,0.010604,0.021526,0.014054,0.024437,0.143584,0.091663,0.018222,0.023444,0.434423,Kraftwerk,Minimum-Maximum,9.0,"Electronic,Rock",https://pitchfork.com/reviews/albums/4557-minimum-maximum/
3,https://pitchfork.com/reviews/albums/11418-going-places-the-august-darnell-years-1976-1983/,0.087805,0.056293,0.029838,0.016234,0.020594,0.023717,0.034106,0.358788,0.028726,0.033581,0.030064,0.018855,0.019831,0.013588,0.021222,0.052132,0.011995,0.013087,0.085314,0.044230,Kid Creole,Going Places: The August Darnell Years 1976-1983,8.6,Pop/R&B,https://pitchfork.com/reviews/albums/11418-going-places-the-august-darnell-years-1976-1983/
4,https://pitchfork.com/reviews/albums/21145-meow-the-jewels/,0.019409,0.087558,0.015504,0.024806,0.055348,0.012996,0.014934,0.050035,0.031041,0.007007,0.008545,0.459265,0.021553,0.022175,0.020627,0.060883,0.028470,0.016778,0.015102,0.027964,Run the Jewels,Meow The Jewels,7.0,Rap,https://pitchfork.com/reviews/albums/21145-meow-the-jewels/


In [113]:
pyLDAvis.prepare(topic_term_dists, 
                 model.get_theta
                 doc_topic_dists, 
                 doc_lengths, dictionary, 
            

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
with torch.no_grad():
    bow = torch.zeros(len(dictionary))
    item = list(zip(*x_bows_train[1])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
    bow[list(item[0])] = torch.tensor(list(item[1])).float()
    topics = model.get_theta(torch.tensor(bow).float().to(device))
    print(f'topic is: {torch.argmax(topics[0])} and text is {x_tokens_train[1]}')

In [116]:
a = model.get_beta().to.
a.shape

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


torch.Size([20, 15048])

In [ ]:
get_most_similar_words(model = model, queries=['pop','cash'], vocabulary = vocab, n_most_similar=5)